## Identifiers

In [1]:
import pickle
import sys
import spacy
from spacy import displacy
import benepar

In [2]:
# Set up and store NLP
nlp = spacy.load("en_core_web_md")
benepar.download('benepar_en3')
nlp.add_pipe('benepar', config={'model': 'benepar_en3'})

with open('./nlp/nlp.pickle', 'wb') as f:
    pickle.dump(nlp, f)

[nltk_data] Downloading package benepar_en3 to
[nltk_data]     C:\Users\regan\AppData\Roaming\nltk_data...
[nltk_data]   Package benepar_en3 is already up-to-date!


In [3]:
# Load NLP
#sys.path.append('../')

with open('./nlp/nlp.pickle', 'rb') as f:
    nlp = pickle.load(f)

In [4]:
from app.classes.spec.primitive import *

from app.src.sentence_summarizer import SentenceSummarizer

summarizer = SentenceSummarizer(nlp)

### Testing it out

In [5]:
from app.src.primitive_identifiers.dict_set_builder import DictSetBuilder

# Will need to put all of this in a config somewhere...
# And will want to get ALL the dict sets for each identifier
# Have it done in a single call
obligation_events = {
    'triggered': 'trip.v.04', 
    'activated': 'activate.v.02', 
    'suspended': 'suspend.v.06', 
    'resumed': 'resume.v.02', 
    'discharged': 'free.v.06', 
    'expired': 'run_out.v.04', 
    'fulfilled': 'satisfy.v.01', 
    'violated': 'transgress.v.01', 
    'terminated': 'end.v.02'
}

dict_set_builder = DictSetBuilder()
dict_set = dict_set_builder.build(obligation_events, 'v')



[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\regan\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [7]:
from app.src.primitive_identifiers.obligation_event_identifier import ObligationEventIdentifier
from app.src.primitive_identifiers.event_scorer import EventScorer
from app.src.primitive_identifiers.variable_scorer import VariableScorer
from app.src.primitive_identifiers.similarity_calculator import SentenceSimilarityCalculator
from app.src.primitive_identifiers.sentence_preprocessor import SentencePreprocessor
from app.templates.meat_sale.templates import meat_sale_templates

event_name_scorer = EventScorer(dict_set)
preprocessor = SentencePreprocessor(nlp)
similarity_calculator = SentenceSimilarityCalculator(preprocessor)
obligation_sent_dict = meat_sale_templates['obligations']
variable_scorer = VariableScorer(nlp, obligation_sent_dict, similarity_calculator)
oe_identifier = ObligationEventIdentifier(event_name_scorer, variable_scorer)


In [15]:

test_sentence = 'seller has resumed delivery of goods'
doc = nlp(test_sentence)

result = oe_identifier.identify(doc)

print(type(result.primitive))
p: ObligationEvent = result.primitive
print('event: ',p.event_name)
print('var:', p.obligation_variable)
print(result.score)

c:\Users\regan\anaconda3\lib\site-packages\torch\distributions\distribution.py:44: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +


<class 'app.classes.spec.sym_event.ObligationEvent'>
event:  resumed
var: delivery
0.797
